In [2]:
include("PowerFlowRes57.jl")
include("case57.m")
E=PF[:,2].*cos((π/180)PF[:,3])+PF[:,2].*sin((π/180)PF[:,3])im
Iₗ=complex(zeros(size(branch,1),2))
for i in 1:size(branch,1)
    Iₗ[i,1]=((PF1[i,4]/baseMVA)+(PF1[i,5]/baseMVA)im)/E[Int(PF1[i,2])]
    Iₗ[i,2]=((PF1[i,6]/baseMVA)+(PF1[i,7]/baseMVA)im)/E[Int(PF1[i,3])]
end

#include("C:/Users/Saba/Desktop/case57.m")
PMU=[1, 4, 6, 9, 15, 20, 24, 28, 30, 32, 36, 38, 41, 46, 50, 53, 57]
r=branch[:,3];x=branch[:,4];y=1./(r+x*im);b=branch[:,5];y₀=b/2*im;
nLinePMU=0
for k in 1:size(branch,1)
    for k′ in 1:size(PMU,1)
        if branch[k,1]==PMU[k′]||branch[k,2]==PMU[k′]
            for k′′ in 1:size(PMU,1)
                if branch[k,1]==PMU[k′]&&branch[k,2]==PMU[k′′]
                    nLinePMU=nLinePMU+1
                end
            end
        end
    end
end
A=zeros(size(branch,1)+nLinePMU,size(bus,1))
Y=complex(zeros(size(branch,1)+nLinePMU,size(branch,1)+nLinePMU));
yₛ=complex(zeros(size(branch,1)+nLinePMU,size(bus,1)));y₀=[y₀;zeros(nLinePMU,1)]
I₎=complex(zeros(size(branch,1)+nLinePMU,1));
a′=0
for k in 1:size(branch,1)
    for k′ in 1:size(PMU,1)
        if branch[k,1]==PMU[k′]||branch[k,2]==PMU[k′]
            if branch[k,1]==PMU[k′]&&branch[k,2]!=PMU[k′]
                A[k+a′,PMU[k′]]=1
                A[k+a′,Int(branch[k,2])]=-1
                Y[k+a′,k+a′]=y[k]
                yₛ[k+a′,PMU[k′]]=y₀[k]
                I₎[k+a′]=Iₗ[k,1]
            elseif branch[k,2]==PMU[k′]&&branch[k,1]!=PMU[k′]
                A[k+a′,PMU[k′]]=1
                A[k+a′,Int(branch[k,1])]=-1
                Y[k+a′,k+a′]=y[k]
                yₛ[k+a′,PMU[k′]]=y₀[k]
                I₎[k+a′]=Iₗ[k,2]
            end
            for k′′ in 1:size(PMU,1)
                if branch[k,1]==PMU[k′]&&branch[k,2]==PMU[k′′]
                    A[k+a′,PMU[k′]]=1
                    A[k+a′,Int(branch[k,2])]=-1
                    Y[k+a′,k+a′]=y[k]
                    yₛ[k+a′,PMU[k′]]=y₀[k]
                    I₎[k+a′]=Iₗ[k,1]
                    A[k+a′+1,PMU[k′′]]=1
                    A[k+a′+1,Int(branch[k,1])]=-1
                    Y[k+a′+1,k+a′+1]=y[k]
                    yₛ[k+a′+1,PMU[k′′]]=y₀[k]
                    I₎[k+a′+1]=Iₗ[k,2]
                    a′=a′+1
                end
            end
        end
    end
end
A′=A
A=A[find(A[i,:]!=zeros(size(A,2)) for i in 1:size(A,1)),:]
Y=Y[find(Y[i,:]!=zeros(size(Y,2)) for i in 1:size(Y,1)),find(Y[i,:]!=zeros(size(Y,2)) for i in 1:size(Y,1))]
yₛ=yₛ[find(A′[i,:]!=zeros(size(A′,2)) for i in 1:size(A′,1)),find(A′[:,i]!=zeros(size(A′,1)) for i in 1:size(A′,2))]
ΙΙ=zeros(size(PMU,1),size(bus,1))
a′′=0
for i in 1:size(bus,1)
    for j in 1:size(PMU,1)
        if i==PMU[j]
            ΙΙ[1+a′′,i]=1
            a′′=a′′+1
        end
    end
end
B=[ΙΙ;Y*A+yₛ]
W=1000*eye(size(B,1))
z=B*E+(2/3000)*B*E.*(rand(size(B,1))+rand(size(B,1))im)
M=(transpose(B)*(W^(-1))*B)^(-1)*transpose(B)*(W^(-1))
x̂=M*z

################
#sqrt(sum((angle(x̂)-angle(E)).^2)/size(B,2)),sqrt(sum((abs(x̂)-abs(E)).^2)/size(B,2))
#sqrt(sum((x̂-E).^2)/size(B,2))
#sqrt(([sum([angle(x̂)-angle(E)]);sum([abs(x̂)-abs(E)])].^2)/size(B,2))
#I₎=I₎[find(I₎)]
#zₜ=[E[PMU];I₎]


57-element Array{Complex{Float64},1}:
  1.04044+0.000511394im
  1.01022-0.0204491im  
 0.980078-0.102318im   
  0.97342-0.124851im   
 0.965617-0.144623im   
 0.969247-0.147381im   
 0.975806-0.129733im   
  1.00238-0.0780539im  
 0.966772-0.162781im   
 0.966849-0.195355im   
 0.959073-0.171982im   
 0.998556-0.18406im    
 0.965164-0.1663im     
         ⋮             
  1.04063-0.203928im   
  1.00898-0.223369im   
  1.00266-0.223925im   
  1.01014-0.231619im   
 0.995516-0.23701im    
  1.02737-0.227987im   
 0.960807-0.195141im   
 0.949353-0.205879im   
 0.975747-0.201937im   
  1.01323-0.1928im     
  0.93008-0.268262im   
 0.924642-0.275824im   

In [2]:
using JuMP

DSE = Model()
NS = 2size(bus,1); T = 24; i = 1:NS; t = 1:T;
@variable(DSE x̂[i,t] )
@variable(DSE γ[i,t] )
@variable(DSE σ̂[i,t] )
α=0.5;β=0.5;
S[i,t-2]=α[i]x̂[i,t-2]+(1-α[i])x̃[i,t-2]
b[i,t-2]=β[i,t](S[i,t-2]-S[i,t-3])+(1-β[i])b[i,t-3]
C[i,t-1]=α[i](1+β[i])
D[i,t-1]=(1+β[i])(1-α[i])x̃[i,t-1]-β[i]*S[i,t-2]+(1-β[i])b[i,t-2]
x̃[i,t]=C[i,t-1]x̂[i,t-1]+D[i,t-1]
σ̃[i,t]=(C[i,t-1]^2)(σ̂[i,t-1]^2)+σ[i,t-1]^2
r̃[i,t]=x̃[i,t]-x̂[i,t]
(R̃[i,t])^2=(1-γ[i,t])(r̃[i,t])^2
M=1000
@objective(DSE Min sum(r̅[i,j,t]^2/σ̅[i,j]^2 for i in 1:NS j in 1:)+sum(R̃[i,t]^2/σ̃[i,t]^2 for i in 1:NS) )
@constraint(DSE rᵐᶥⁿ[i](1-γ[i,t]) <= R̃[i,t] <= rᵐᵃˣ[i](1-γ[i,t]) )
@constraint(DSE r̃[i,t]-rᵐᵃˣ[i]γ[i,t] <= R̃[i,t] <= r̃[i,t]-rᵐᶥⁿ[i]γ[i,t] )
@constraint(DSE (r̃[i,t]-3σ̂[i,t])/M[i] <= γ[i,t] <= (r̃[i,t]-3σ̂[i,t])/M[i]+1 )
@constraint(DSE r̅[i,j,t] <= x̅[i,j,t]-x̂[i,t] )
@constraint(DSE r̅[i,j,t] <= x̂[i,t]-x̅[i,j,t] )
@constraint(DSE r̃[i,t] <= x̃[i,t]-x̂[i,t] )
@constraint(DSE r̃[i,t] <= x̂[i,t]-x̃[i,t] )
@constraint(DSE σ̂[i,t] == γ[i,t]/√sum(1/σ̅[i,j]^2 for j in 1:)+(1-γ[i,t])/√(sum(1/σ̅[i,j]^2 for j in 1:)+1/σ̃[i,t]^2) )
print(DSE)
#status = solve(DSE)
println("Objective value: " getobjectivevalue(DSE))
println("x̂[i,t] = " getvalue(x̂[i,t]))
println("γ[i,t] = " getvalue(γ[i,t]))
println("σ̂[i,t] = " getvalue(σ̂[i,t]))


INFO: Recompiling stale cache file C:\Users\Saba\.julia\lib\v0.5\JuMP.ji for module JuMP.


LoadError: syntax: missing comma or ) in argument list